# Automatic predictor

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

## Dataset

In [2]:
class MetricsCorrelationDataset(Dataset):

    def __init__(self, texts, summaries, labels, tokenizer, max_length):
        self.texts = texts
        self.summaries = summaries
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        label = self.labels[idx]
        text_encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        summary_encoding = self.tokenizer(summary, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        encoding = self.tokenizer(text, summary, truncation='only_first', max_length=self.max_length)
        ans = {
            # 'text_input_ids': text_encoding['input_ids'].flatten(), 
            # 'text_attention_mask': text_encoding['attention_mask'].flatten(), 
            # 'summary_input_ids': summary_encoding['input_ids'].flatten(),
            # 'summary_attention_mask': summary_encoding['attention_mask'].flatten(),
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'label': torch.tensor(label),
            # 'labels': torch.tensor(label)
        }
        
        return ans

## Model

## Data

In [3]:
import pandas as pd
import numpy as np

index = "Ind"
title = "title"
article = "text"
ground_truth = "summary"

files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]
summaries_fields = []
human_metrics = [
    "Актуальность",
    "Последовательность",
    "Беглость",
    "Согласованность",
    "Комментарий"
]

for file in files:
    model_name = file.split(".")[0]
    summaries_fields.append(model_name)
    for metric in human_metrics:
        summaries_fields.append(f"{model_name}_{metric}")

summaries_fields_types = {field : ('Float64' if field.split("_")[-1] != human_metrics[-1] and field + ".txt" not in files and field + ".csv" not in files else str) for field in summaries_fields}
print(summaries_fields_types)
summaries_fields_types[index] = 'Int64'
summaries_fields_types[article] = summaries_fields_types[ground_truth] = str

metrics_data = pd.read_csv("metrics_data.csv", dtype=summaries_fields_types)
expert_data = pd.read_csv("compiled_expert_data.csv", dtype=summaries_fields_types)
# data = pd.read_csv("export_data.csv")

{'mbart_predictions': <class 'str'>, 'mbart_predictions_Актуальность': 'Float64', 'mbart_predictions_Последовательность': 'Float64', 'mbart_predictions_Беглость': 'Float64', 'mbart_predictions_Согласованность': 'Float64', 'mbart_predictions_Комментарий': <class 'str'>, 'mt5_predictions': <class 'str'>, 'mt5_predictions_Актуальность': 'Float64', 'mt5_predictions_Последовательность': 'Float64', 'mt5_predictions_Беглость': 'Float64', 'mt5_predictions_Согласованность': 'Float64', 'mt5_predictions_Комментарий': <class 'str'>, 'summarunner_predictions': <class 'str'>, 'summarunner_predictions_Актуальность': 'Float64', 'summarunner_predictions_Последовательность': 'Float64', 'summarunner_predictions_Беглость': 'Float64', 'summarunner_predictions_Согласованность': 'Float64', 'summarunner_predictions_Комментарий': <class 'str'>, 'llama_7b_predictions': <class 'str'>, 'llama_7b_predictions_Актуальность': 'Float64', 'llama_7b_predictions_Последовательность': 'Float64', 'llama_7b_predictions_Бегло

In [4]:
metrics_data.head()

,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions,...,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor,llama_7b_predictions_meteor,starling_predictions_meteor,yagpt_predictions_meteor,yagpt3_predictions_meteor,mbart_predictions_bleu.1,mt5_predictions_bleu.1,summarunner_predictions_bleu.1
0,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,- Постоянно включенный Bluetooth на смартфоне ...,Доцент кафедры информатики РЭУ им. Плеханова А...,...,0.126459,0.053648,0.168245,0.207257,0.127202,0.168304,0.139373,NaN,NaN,NaN
1,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",- Член президиума координационного совета оппо...,"Мария Колесникова, член президиума координацио...",...,0.248829,0.101734,0.263807,0.231588,0.310913,0.327753,0.224195,NaN,NaN,NaN
2,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",- Аналитики Deutsche Bank предсказали наступле...,Аналитики Deutsche Bank предсказали наступлени...,...,0.309631,0.086664,0.314012,0.262029,0.238650,0.295642,0.311487,NaN,NaN,NaN
3,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,ГИБДД и Минтранс разработали проект изменений ...,- ГИБДД и Минтранс разработали масштабный прое...,ГИБДД совместно с Минтрансом разработали масшт...,...,0.306315,0.082645,0.233184,0.256739,0.270406,0.338463,0.296722,NaN,NaN,NaN
4,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,"В ходе конференции Tesla Battery Day, Илон Мас...",- Глава Tesla Илон Маск анонсировал вывод на р...,На конференции Battery Day Илон Маск объявил о...,...,0.098891,0.100573,0.146562,0.227205,0.193430,0.202198,0.251442,NaN,NaN,NaN


## Model

## Data

In [5]:
import math

human_metrics = human_metrics[:-1]

models = {
    "mbart_predictions",
    "mt5_predictions",
    "summarunner_predictions",
    "llama_7b_predictions",
    "starling_predictions",
    "yagpt_predictions",
    "yagpt3_predictions"
}
metrics = {
    "bleu",
    "rouge1",
    "meteor",
    "bertscore_f1"
}

texts = []
summaries = []
labels = {k: {"bleu": [], "bertscore": [], "rouge": [], "meteor": []} for k in [0.25, 0.5, 0.75]}
human_scores = []
auto_scores = {"bleu": [], "bertscore": [], "rouge": [], "meteor": []}
totals = {"bleu": 0, "bertscore": 0, "rouge": 0, "meteor": 0}
deviations = {"bleu": [], "bertscore": [], "rouge": [], "meteor": []}


for model in models:
    for (mteric_index, metric_row), (expert_index, expert_row)  in zip(metrics_data.iterrows(), expert_data.iterrows()):
        # if any([row[f"{model}_{metric}"] is None or math.isnan(row[f"{model}_{metric}"]) for metric in metrics]) or row[model] is None or type(row[model]) != str:
        #     continue
        texts.append(metric_row["summary"])
        summaries.append(metric_row[model])
        human_scores.append(np.mean([expert_row[f"{model}_{metric}"] for metric in human_metrics]) / 5)
        # print(model, human_scores[-1])

        auto_scores["bleu"].append(metric_row[f"{model}_bleu"])
        auto_scores["rouge"].append(metric_row[f"{model}_rouge1"])
        auto_scores["meteor"].append(metric_row[f"{model}_meteor"])
        auto_scores["bertscore"].append(metric_row[f"{model}_bertscore_f1"])

mean_human = np.mean(human_scores)
mean_bleu = np.mean(auto_scores["bleu"])
mean_rouge = np.mean(auto_scores["rouge"])
mean_meteor = np.mean(auto_scores["meteor"])
mean_bertscore = np.mean(auto_scores["bertscore"])

for i in range(len(texts)):
    human_scores[i] -= mean_human
    auto_scores["bleu"][i] -= mean_bleu
    auto_scores["rouge"][i] -= mean_rouge
    auto_scores["meteor"][i] -= mean_meteor
    auto_scores["bertscore"][i] -= mean_bertscore

    deviations["bleu"].append(abs(auto_scores["bleu"][i] - human_scores[i]))
    deviations["rouge"].append(abs(auto_scores["rouge"][i] - human_scores[i]))
    deviations["meteor"].append(abs(auto_scores["meteor"][i] - human_scores[i]))
    deviations["bertscore"].append(abs(auto_scores["bertscore"][i] - human_scores[i]))

print("Median devs:")
print("\tBLEU: ", np.median(deviations["bleu"]), np.max(deviations["bleu"]))
print("\tROUGE: ", np.median(deviations["rouge"]), np.max(deviations["rouge"]))
print("\tMETEOR: ", np.median(deviations["meteor"]), np.max(deviations["meteor"]))
print("\tBERTSCORE: ", np.median(deviations["bertscore"]), np.max(deviations["bertscore"]))

labels["bleu"] = [1.0 if deviations["bleu"][i] < np.quantile(deviations["bleu"], 0.5) else 0.0 for i in range(len(texts))]
labels["rouge"] = [1.0 if deviations["rouge"][i] < np.quantile(deviations["rouge"], 0.5) else 0.0 for i in range(len(texts))]
labels["meteor"] = [1.0 if deviations["meteor"][i] < np.quantile(deviations["meteor"], 0.5) else 0.0 for i in range(len(texts))]
labels["bertscore"] = [1.0 if deviations["bertscore"][i] < np.quantile(deviations["bertscore"], 0.5) else 0.0  for i in range(len(texts))]

Median devs:
	BLEU:  0.08652315462924738 0.5889453306820962
	ROUGE:  0.08388135593220336 0.69087546980561
	METEOR:  0.0933422032178387 0.5547894310323825
	BERTSCORE:  0.08324829339981082 0.510270966206278


In [6]:
print(len(texts), len(summaries), len(labels["bleu"]))

700 700 700


## Dataset instance

In [7]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

def get_dataset(tokenizer, metric_name):
    texts_len = len(texts)
    train = int(texts_len * 0.9)
    test_val = int(texts_len * 0.1)
    max_length = None
    return {
        "train": MetricsCorrelationDataset(texts=texts[0:train], summaries=summaries[0:train], labels=labels[metric_name][0:train], tokenizer=tokenizer, max_length=max_length),
        "test": MetricsCorrelationDataset(texts=texts[train:texts_len], summaries=summaries[train:texts_len], labels=labels[metric_name][train:texts_len], tokenizer=tokenizer, max_length=max_length),
        #"val": MetricsCorrelationDataset(texts=texts[train + test_val:train + 2 * test_val], summaries=summaries[train + test_val:train + 2 * test_val], labels=labels[metric_name][train + test_val:train + 2 * test_val], tokenizer=tokenizer, max_length=max_length)
    }

## Trainer

In [8]:
from transformers import TrainingArguments
from transformers import Trainer

In [9]:
training_args = TrainingArguments(
    output_dir="automatic_predictor",
    learning_rate=3e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=25,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

## Work

In [10]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize
import numpy as np

def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    probabilities = np.exp(predictions) / np.sum(np.exp(predictions), axis=-1, keepdims=True)
    predictions = torch.tensor([float(round(x)) for x in predictions.flatten()])
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="macro")
    p = precision_score(labels, predictions, average="macro")
    r = recall_score(labels, predictions, average="macro")
    return {"precision": p, "recall": r, "f1": f1, "accuracy": accuracy}

In [11]:
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptEncoderConfig,
)
peft_config = PromptEncoderConfig(task_type="SEQ_CLS", num_virtual_tokens=30, encoder_hidden_size=256)

### Rouge

In [12]:
from transformers import AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split



model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruRoberta-large", num_labels=1)
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")
# tokenizer.model_max_length=484

rouge_dataset = get_dataset(tokenizer, "rouge")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=rouge_dataset["train"],
    eval_dataset=rouge_dataset["test"],
    tokenizer=rouge_dataset["train"].tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/skatori/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch,Training Loss,Validation Loss


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
model

In [ ]:
import gc
import torch

model = None
tokenizer = None
gc.collect()
torch.cuda.empty_cache() 

### Bertscore

In [ ]:
from transformers import AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split



model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruRoberta-large", num_labels=1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")
tokenizer.model_max_length=492

rouge_dataset = get_dataset(tokenizer, "bertscore")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=rouge_dataset["train"],
    eval_dataset=rouge_dataset["test"],
    tokenizer=rouge_dataset["train"].tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
import gc
import torch

model = None
tokenizer = None
gc.collect()
torch.cuda.empty_cache() 

### METEOR

In [ ]:
from transformers import AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split



model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruRoberta-large", num_labels=1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")
tokenizer.model_max_length=492

rouge_dataset = get_dataset(tokenizer, "meteor")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=rouge_dataset["train"],
    eval_dataset=rouge_dataset["test"],
    tokenizer=rouge_dataset["train"].tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
import gc
import torch

model = None
tokenizer = None
gc.collect()
torch.cuda.empty_cache() 